In [2]:
#ライブラリのインポート
from bs4 import BeautifulSoup
import re #不要文字削除
import requests
from time import sleep
import pandas as pd
import gspread 
from oauth2client.service_account import ServiceAccountCredentials

In [3]:
# 最後のページの数値を取得
#引用先のURL（東京駅まで電車で20分以内、最寄駅から徒歩5分以内、家賃25万以内、2K/2DK/2LDKで検索）
url = "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&ta=13&bs=040&ekInput=25620&tj=20&nk=-1&ct=25.0&cb=0.0&md=05&md=06&md=07&et=5&mt=9999999&mb=0&cn=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&pc=30"
res = requests.get(url)
res.encoding = 'utf-8'
soup = BeautifulSoup(res.text, 'html.parser') #取得したHTMLをBeautifulSoupで解析
page_elements = soup.select("ol.pagination-parts a")
page_numbers = [int(element.text) for element in page_elements]
last_page = max(page_numbers)
print(last_page)

9


In [4]:
#空のリストを作成
data_list = []

In [5]:
# 正常にHTML情報が取得できるか確認してからデータ取得開始
if res.status_code == 200:

    #全ページのデータ取得
    for page in range(1, last_page +1): 
        target_url = url.format(page)
        res = requests.get(target_url) #requestを使ってURLにアクセス
        sleep(1) #相手サイトの負荷軽減
        res.encoding = 'utf-8' #文字化け防止
        soup = BeautifulSoup(res.text, 'html.parser') #取得したHTMLをBeautifulSoupで解析
        contents = soup.find_all('div', class_= 'cassetteitem') #全ての物件情報取得

        #物件・部屋情報の取得
        for content in contents:
            detail = content.find('div', class_='cassetteitem-detail') #物件情報
            table = content.find('table', class_='cassetteitem_other') #部屋情報

            #物件情報から必要情報を取得
            name = detail.find('div', class_='cassetteitem_content-title').text
            address = detail.find('li', class_='cassetteitem_detail-col1').text

            #部屋情報を取得
            tr_tags = table.find_all('tr', class_='js-cassette_link')
            for tr_tag in tr_tags:
                #部屋情報から必要情報を取得
                floor, price, first_fee, capacity = tr_tag.find_all('td')[2:6]
                #さらに細かい情報取得
                rent, administration = price.find_all('li')
                deposit, gratuity = first_fee.find_all('li')
                madori, menseki = capacity.find_all('li')
                #取得した全ての情報を辞書に格納
                data = {
                    'name' : name,
                    'address' : address,
                    'floor': floor.text,
                    'rent' : rent.text,
                    'administration' : administration.text,
                    'deposit' : deposit.text,
                    'gratuity' : gratuity.text,
                    'madori' : madori.text,
                    'menseki' : menseki.text
                }
                #取得した辞書を格納
                data_list.append(data)

In [6]:
#データフレームを作成
df = pd.DataFrame(data_list)

In [7]:
df.dtypes #データ型確認

name              object
address           object
floor             object
rent              object
administration    object
deposit           object
gratuity          object
madori            object
menseki           object
dtype: object

In [8]:
#データ確認
df.head()

,name,address,floor,rent,administration,deposit,gratuity,madori,menseki
0,レジディア神田淡路町,東京都千代田区神田淡路町２,\r\n\t\t\t\t\t\t\t\t\t\t\t8階,23万円,15000円,23万円,23万円,2DK,41.8m2
1,デュオ・スカーラ御茶ノ水II,東京都千代田区神田淡路町１,\r\n\t\t\t\t\t\t\t\t\t\t\t3階,15.5万円,10000円,15.5万円,15.5万円,2K,35.36m2
2,東京メトロ丸ノ内線 淡路町駅 8階建 築20年,東京都千代田区神田須田町１,\r\n\t\t\t\t\t\t\t\t\t\t\t6階,14万円,10000円,14万円,14万円,2DK,36.88m2
3,内木ビル,東京都墨田区江東橋４,\r\n\t\t\t\t\t\t\t\t\t\t\t4階,9万円,-,9万円,-,2DK,35m2
4,東京メトロ半蔵門線 錦糸町駅 4階建 築57年,東京都墨田区江東橋４,\r\n\t\t\t\t\t\t\t\t\t\t\t4階,9万円,-,9万円,-,2DK,37.26m2


In [11]:
#DB全体の不要な文字を取り除く
def remove_unwanted_chars(text):
    if isinstance(text, str):
        return re.sub('[\n\r\t]', '', text)
    return text

# データフレームのすべての要素に関数を適用
df = df.applymap(remove_unwanted_chars)

In [12]:
df['floor'] = df['floor'].str.replace('階', '') #floorのデータの表現を統一

In [13]:
#お金（rent, administration	, deposit, gratuity）を整数型に変換
def yen_to_int(text):
    try:
        text = str(text)

        if '-' in text:
            amount = 0
        elif '万円' in text:
            amount = float(text.replace('万円', '')) * 10000
        else:
            amount = float(text.replace('円', ''))
    except ValueError:
        amount = 0 # 形式に一致しない場合は0を返す
    return int(amount)

df['rent'] = df['rent'].apply(yen_to_int)
df['administration'] = df['administration'].apply(yen_to_int)
df['deposit'] = df['deposit'].apply(yen_to_int)
df['gratuity'] = df['gratuity'].apply(yen_to_int)

In [14]:
#mensekiのデータ（比例尺度）に変換
df['menseki'] = df['menseki'].str.replace('m2','').astype(float)

df.head()

,name,address,floor,rent,administration,deposit,gratuity,madori,menseki
0,レジディア神田淡路町,東京都千代田区神田淡路町２,8,230000,15000,230000,230000,2DK,41.80
1,デュオ・スカーラ御茶ノ水II,東京都千代田区神田淡路町１,3,155000,10000,155000,155000,2K,35.36
2,東京メトロ丸ノ内線 淡路町駅 8階建 築20年,東京都千代田区神田須田町１,6,140000,10000,140000,140000,2DK,36.88
3,内木ビル,東京都墨田区江東橋４,4,90000,0,90000,0,2DK,35.00
4,東京メトロ半蔵門線 錦糸町駅 4階建 築57年,東京都墨田区江東橋４,4,90000,0,90000,0,2DK,37.26


In [15]:
# 特定の列に基づいて重複を確認
duplicate_rows = df[df.duplicated(subset=['address',  'floor', 'rent', 'menseki'])]

# 重複件数の表示
print(f"重複件数: {duplicate_rows.shape[0]}")

# 重複データの表示
print(duplicate_rows)

重複件数: 339
                         name        address floor    rent  administration  \
6     東京メトロ丸ノ内線 淡路町駅 12階建 築9年  東京都千代田区神田淡路町２     8  230000           15000   
8        都営三田線 水道橋駅 16階建 築54年      東京都文京区本郷１    10  198000           12000   
15      ＪＲ総武線快速 錦糸町駅 8階建 築46年     東京都墨田区江東橋２     5   93000               0   
19                   パークサイド関口      東京都墨田区錦糸４     7  110000               0   
23      都営浅草線 東日本橋駅 11階建 築41年    東京都中央区東日本橋２     7  195000               0   
..                        ...            ...   ...     ...             ...   
364                      柏倉ビル    東京都荒川区東日暮里５     3  110000               0   
365               シャルムコート東日本橋    東京都中央区東日本橋２     6  228000           18000   
366   東京メトロ丸ノ内線 淡路町駅 13階建 築7年  東京都千代田区神田美土代町     7  160000           10000   
367  東京メトロ日比谷線 八丁堀駅 12階建 築19年      東京都中央区新川２     3  179000           16000   
368  東京メトロ丸ノ内線 淡路町駅 12階建 築16年  東京都千代田区神田淡路町１     6  166000           10000   

     deposit  gratuity madori  menseki  
6     230000

In [16]:
# 重複データの削除
df.drop_duplicates(subset=['address', 'floor', 'rent', 'menseki'], inplace=True)

In [19]:
#スプレッドシートに転記
import gspread 
from oauth2client.service_account import ServiceAccountCredentials
SP_CREDENTIAL_FILE = "tech0-step3-suumo-data-27327b44e1bb.json"
SP_COPE = [
    "https://www.googleapis.com/auth/drive",
    "https://spreadsheets.google.com/feeds"
]
SP_SHEET_ID = "1zWKnfAq6sBT4kjeWJ42W-5yBGeevcisgS6yPUph1qw8"
SP_SHEET="suumo_lowdata"

credentials = ServiceAccountCredentials.from_json_keyfile_name(SP_CREDENTIAL_FILE, SP_COPE)
gc = gspread.authorize(credentials)

sh = gc.open_by_key(SP_SHEET_ID)
worksheet = sh.worksheet(SP_SHEET)

worksheet.clear()

worksheet.update([df.columns.values.tolist()] + df.values.tolist())

C:\Users\user\AppData\Local\Temp\ipykernel_8104\4152035202.py:20: DeprecationWarning: [Deprecated][in version 6.0.0]: Method signature's arguments 'range_name' and 'values' will change their order. We recommend using named arguments for minimal impact. In addition, the argument 'values' will be mandatory of type: 'List[List]'. (ex) Worksheet.update(values = [[]], range_name=) 
  worksheet.update([df.columns.values.tolist()] + df.values.tolist())


{'spreadsheetId': '1zWKnfAq6sBT4kjeWJ42W-5yBGeevcisgS6yPUph1qw8',
 'updatedRange': 'suumo_lowdata!A1:I31',
 'updatedRows': 31,
 'updatedColumns': 9,
 'updatedCells': 279}